In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
responses_df = pd.read_csv(r"augmentedform (1).csv")
print(responses_df.shape)
responses_df.head()

(147, 15)


,What do you enjoy doing in your free time?,Consider a project where you have a tight deadline. How do you balance the need to meet that deadline with your commitment to thoroughness and precision?,"Imagine yourself as a game referee. You have made arrived at a decision that you feel is correct. However, one team feels that the decision is incorrect and persistently argues to overturn the decision. How would you proceed in such a situation?","How do you feel about exploring new ideas and concepts, even if they are unconventional?","What workplace do you prefer, and do you prefer working alone or as part of a team and why?","Imagine you are in a situation where you have been offered huge opportunity abroad, however it will be a completely new place and you will not know anyone there. You also have an opportunity in your residing city, which offers less but provides familiarity and comfortable surroundings, which one would you chose and why?",Can you tell how often you feel anxious or nervous in a social situation and how did you handle those situations?,"How do you handle criticism or feedback, especially when it's not delivered in the most constructive way?","Imagine waking up feeling extremely frustrated and irritated for no apparent reason. Early in the morning itself you encounter various challenges and setbacks, such as traffic congestion, a difficult conversation with a colleague, and a technology malfunction. How would your rest of the day go?",How would you describe your work-life balance?,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,"Sleeping, watching stuff, instagram",Prioritise and focus,I'd ask them to approach higher authorities if...,Yeah they should be explored. No room for regr...,"Outdoor workplace, as part of a team",Move abroad,"Very often, deep breaths and calling someone f...",Not take it personally,I'd make sure it goes better,Not aware,75.0,46.0,46.0,69.0,67.0
1,"Listening to music, Reading books",by time management,I would listen to their arguments and only ove...,I mostly find it interesting to explore new id...,I prefer working as a team but I like it bette...,The opportunity in my residing city,I haven't felt much like this lately. In occas...,I do not handle criticism well unless it is pu...,The next few hours of the day might go bad as ...,I would rate it 8/10.,83.0,87.5,27.0,87.5,15.0
2,Listening to music,I make small targets and try to achieve them w...,I would listen to the other side as well and t...,I like exploring new ideas and concepts.,I prefer working in a team because I feel it g...,I will prefer going abroad that is offering me...,I do not often feel anxious in social situatio...,"At first, I feel a bit bad, but then I ponder ...",My rest of the day will go a bit off.,I divide my time between my work and my person...,83.0,75.0,46.0,79.0,56.0
3,Gaming,Use online resources like AI tools to collect ...,Explain the situation to the team and stick to...,Always intrigued in exploring new ideas,Hybrid. Prefer working alone during initial st...,Would choose new place considering the work li...,I do feel anxious. Try to calm myself down and...,Try to understand their point of view. If the ...,"I would try to make sure I stay calm, and not ...",Currently a student so it's quite even,79.0,56.0,67.0,71.0,27.0
4,Watching TV or dancing or learn something new,I always make a list of things I have to do an...,"I will think about it, and ask the Thursday um...",Excited,Team because more ideas and more work can be d...,Abroad,I feel nervous in social situations. To handle...,I ignore it.,Bad,When I am too busy I forget about my family. A...,52.0,77.0,21.0,60.0,92.0


In [ ]:
n_questions = 10
answer_df = pd.DataFrame()
score_df = pd.DataFrame()
answer_df['Answers'] = responses_df.iloc[:, :10].values.ravel()
# score_df = pd.concat([] * n_questions, ignore_index=True)
columns_to_repeat = responses_df.columns[10:]
score_df[columns_to_repeat] = np.repeat(responses_df[columns_to_repeat].values, 10, axis=0)
score_df=score_df/20.0
df = pd.concat([answer_df, score_df], axis=1)
df

,Answers,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,"Sleeping, watching stuff, instagram",3.75,2.30,2.3,3.45,3.35
1,Prioritise and focus,3.75,2.30,2.3,3.45,3.35
2,I'd ask them to approach higher authorities if...,3.75,2.30,2.3,3.45,3.35
3,Yeah they should be explored. No room for regr...,3.75,2.30,2.3,3.45,3.35
4,"Outdoor workplace, as part of a team",3.75,2.30,2.3,3.45,3.35
...,...,...,...,...,...,...
1465,opportunity abroad as it gives room for growth.,4.70,4.15,3.0,4.15,1.25
1466,"Occasionally, I deal with them by monitoring m...",4.70,4.15,3.0,4.15,1.25
1467,by revisiting the task.,4.70,4.15,3.0,4.15,1.25
1468,I find it challenging but manage to persevere.,4.70,4.15,3.0,4.15,1.25


In [ ]:
import re
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import contractions
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
df["Answers"] = df["Answers"].apply(lambda text: text.lower().strip())
df["Answers"] = df["Answers"].apply(lambda text: contractions.fix(text))
df["Answers"] = df["Answers"].apply(lambda text: re.sub(r"[^\w\s.,]", "", text))
df["Answers"] = df["Answers"].apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in text.split()]))

In [ ]:
X = df['Answers']
# X = df.drop(['Openness ', 'Conscientiousness ', 'Extraversion ', 'Agreeableness ', 'Neuroticism '], axis = 1)
y_O = df['Openness ']
y_C = df['Conscientiousness ']
y_E = df['Extraversion ']
y_A = df['Agreeableness ']
y_N = df['Neuroticism ']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0)

# BERT (Base)

In [ ]:
# Dont run this again and again
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess = hub.KerasLayer(preprocess_url, trainable=True)
bert_encoder = hub.KerasLayer(encoder_url, trainable=True)

In [ ]:
rmse = tf.keras.metrics.RootMeanSquaredError()
tf.random.set_seed(42)
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_1")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_2")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_3")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_4")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_5")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_6")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_7")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_8")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_9")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_10")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_11")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_12")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_13")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_14")(layer)
layer = tf.keras.layers.Dense(1, activation='linear', name="output")(layer)
model_O = tf.keras.Model(inputs=[text_input], outputs = [layer])
model_O.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse,'mae'])
model_O.summary()

ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_text[0][0]']          
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [ ]:
Epochs = 20
results_O = model_O.fit(X, y_O, epochs=Epochs)

Epoch 1/20
5/5 [==============================] - 4s 781ms/step - loss: 1257.9642 - root_mean_squared_error: 35.4678 - mae: 29.1613
Epoch 2/20
5/5 [==============================] - 4s 818ms/step - loss: 90.8730 - root_mean_squared_error: 9.5327 - mae: 8.8429
Epoch 3/20
5/5 [==============================] - 4s 785ms/step - loss: 180.3344 - root_mean_squared_error: 13.4289 - mae: 13.2315
Epoch 4/20
5/5 [==============================] - 4s 796ms/step - loss: 194.1676 - root_mean_squared_error: 13.9344 - mae: 13.7386
Epoch 5/20
5/5 [==============================] - 4s 830ms/step - loss: 188.3299 - root_mean_squared_error: 13.7233 - mae: 13.5232
Epoch 6/20
5/5 [==============================] - 4s 799ms/step - loss: 159.6889 - root_mean_squared_error: 12.6368 - mae: 12.4055
Epoch 7/20
5/5 [==============================] - 4s 800ms/step - loss: 74.9958 - root_mean_squared_error: 8.6600 - mae: 7.9520
Epoch 8/20
5/5 [==============================] - 4s 830ms/step - loss: 25.3954 - root_m

In [ ]:
model_O.evaluate(X,y_O)

5/5 [==============================] - 3s 293ms/step - loss: 5.4304 - root_mean_squared_error: 2.3303 - mae: 1.8621


[5.4303765296936035, 2.3303167819976807, 1.8621221780776978]

# BERT (Large)

In [ ]:
# Dont run this again and again
preprocess_url_large = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url_large = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4"
bert_preprocess_large = hub.KerasLayer(preprocess_url_large)
bert_encoder_large = hub.KerasLayer(encoder_url_large)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# Define a learning rate schedule (polynomial decay)
initial_learning_rate = 2e-3
end_learning_rate = 1e-6
decay_steps = 10000
lr_schedule = PolynomialDecay(
    initial_learning_rate, decay_steps, end_learning_rate, power=1
)

# Define the RMSE metric
rmse = tf.keras.metrics.RootMeanSquaredError()

# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text')

# Preprocess the text using the large BERT model
preprocessed_text = bert_preprocess_large(text_input)
outputs = bert_encoder_large(preprocessed_text)

# Adjust the BERT output by adding more dense layers
layer = tf.keras.layers.Dense(1024, activation='relu', name="dense_1")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(1024, activation='relu', name="dense_2")(layer)

layer = tf.keras.layers.Dense(512, activation='relu', name="dense_3")(layer)
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_4")(layer)

# Add task-specific layers (customize for your specific task)
layer = tf.keras.layers.Dense(256, activation='relu', name="task_dense_1")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="task_dense_2")(layer)

layer = tf.keras.layers.Dense(128, activation='relu', name="task_dense_3")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="task_dense_4")(layer)

layer = tf.keras.layers.Dense(64, activation='relu', name="task_dense_5")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="task_dense_6")(layer)

# Output layer for regression task (change as needed for your task)
output_layer = tf.keras.layers.Dense(1, activation='linear', name="output")(layer)

# Create the model
large_model_optimized = tf.keras.Model(inputs=[text_input], outputs=[output_layer])

# Compile the model with a custom learning rate schedule
large_model_optimized.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                               loss='mean_squared_error',
                               metrics=[rmse, 'mae'])

# Display the model summary
large_model_optimized.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_word_ids': (None,    0         ['input_text[0][0]']          
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [ ]:
Epochs = 20
large_results_O = large_model_optimized.fit(X, y_O, epochs=Epochs)

Epoch 1/20
5/5 [==============================] - 25s 890ms/step - loss: 122.5075 - root_mean_squared_error: 11.0683 - mae: 10.4041
Epoch 2/20
5/5 [==============================] - 5s 927ms/step - loss: 67.8074 - root_mean_squared_error: 8.2345 - mae: 6.8621
Epoch 3/20
5/5 [==============================] - 5s 923ms/step - loss: 50.7801 - root_mean_squared_error: 7.1260 - mae: 6.2417
Epoch 4/20
5/5 [==============================] - 5s 931ms/step - loss: 20.3710 - root_mean_squared_error: 4.5134 - mae: 3.8501
Epoch 5/20
5/5 [==============================] - 5s 955ms/step - loss: 15.2519 - root_mean_squared_error: 3.9054 - mae: 3.2473
Epoch 6/20
5/5 [==============================] - 5s 950ms/step - loss: 13.8613 - root_mean_squared_error: 3.7231 - mae: 3.0632
Epoch 7/20
5/5 [==============================] - 5s 969ms/step - loss: 9.0008 - root_mean_squared_error: 3.0001 - mae: 2.4417
Epoch 8/20
5/5 [==============================] - 5s 948ms/step - loss: 6.6549 - root_mean_squared_er

In [ ]:
large_model_optimized.evaluate(X,y_O)

5/5 [==============================] - 7s 891ms/step - loss: 5.5253 - root_mean_squared_error: 2.3506 - mae: 1.8141


[5.525299549102783, 2.350595474243164, 1.8140591382980347]

In [ ]:
print("46/46 [==============================] - 62s 925ms/step - loss: 4691.8633 - root_mean_squared_error: 68.4972 - mae: 67.4045")

46/46 [==============================] - 62s 925ms/step - loss: 4691.8633 - root_mean_squared_error: 68.4972 - mae: 67.4045


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers

# Define a learning rate schedule (polynomial decay)
initial_learning_rate = 2e-3
end_learning_rate = 1e-6
decay_steps = 10000
lr_schedule = PolynomialDecay(
    initial_learning_rate, decay_steps, end_learning_rate, power=1
)

# Define the RMSE metric
rmse = tf.keras.metrics.RootMeanSquaredError()

# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text')

# Preprocess the text using the large BERT model
preprocessed_text = bert_preprocess_large(text_input)
outputs = bert_encoder_large(preprocessed_text)

# Adjust the BERT output by adding LSTM layers with regularization
layer = tf.keras.layers.Reshape((-1, 1))(outputs['pooled_output'])
layer = tf.keras.layers.LSTM(1024, return_sequences=True, kernel_regularizer=regularizers.l2(0.1), name="lstm_1")(layer)
layer = tf.keras.layers.LSTM(1024, return_sequences=True, kernel_regularizer=regularizers.l2(0.01), name="lstm_2")(layer)

layer = tf.keras.layers.LSTM(512, return_sequences=True, kernel_regularizer=regularizers.l2(0.1), name="lstm_3")(layer)
layer = tf.keras.layers.LSTM(512, return_sequences=True, kernel_regularizer=regularizers.l2(0.01), name="lstm_4")(layer)

# Add task-specific layers with regularization (customize for your specific task)
layer = tf.keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.1), name="lstm_5")(layer)
layer = tf.keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.01), name="lstm_6")(layer)

layer = tf.keras.layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(0.1), name="lstm_7")(layer)
layer = tf.keras.layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01), name="lstm_8")(layer)


# Output layer for regression task (change as needed for your task)

output_layer = tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.01), name="output")(layer)

# Create the model
large_model_lstm_optimized = tf.keras.Model(inputs=[text_input], outputs=[output_layer])

# Compile the model with a custom learning rate schedule
large_model_lstm_optimized.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                               loss='mean_squared_error',
                               metrics=[rmse, 'mae'])

# Display the model summary
large_model_lstm_optimized.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_word_ids': (None,    0         ['input_text[0][0]']          
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [ ]:
Epochs = 20
large_results_lstm_O = large_model_lstm_optimized.fit(X, y_O, epochs=Epochs)

Epoch 1/20
5/5 [==============================] - 31s 2s/step - loss: 399.7011 - root_mean_squared_error: 13.6539 - mae: 13.4236
Epoch 2/20
5/5 [==============================] - 13s 3s/step - loss: 219.1440 - root_mean_squared_error: 9.9275 - mae: 9.6120
Epoch 3/20
5/5 [==============================] - 13s 2s/step - loss: 124.0197 - root_mean_squared_error: 7.5166 - mae: 7.0744
Epoch 4/20
5/5 [==============================] - 12s 2s/step - loss: 74.2418 - root_mean_squared_error: 5.8042 - mae: 5.2747
Epoch 5/20
5/5 [==============================] - 12s 2s/step - loss: 50.9932 - root_mean_squared_error: 4.7820 - mae: 4.1645
Epoch 6/20
5/5 [==============================] - 12s 2s/step - loss: 38.9594 - root_mean_squared_error: 4.1127 - mae: 3.4626
Epoch 7/20
5/5 [==============================] - 12s 2s/step - loss: 31.2615 - root_mean_squared_error: 3.6304 - mae: 2.9924
Epoch 8/20
5/5 [==============================] - 12s 2s/step - loss: 25.2439 - root_mean_squared_error: 3.2293 -

In [ ]:
large_model_lstm_optimized.evaluate(X,y_O)

5/5 [==============================] - 12s 1s/step - loss: 8.8924 - root_mean_squared_error: 2.4592 - mae: 1.8887


[8.892436027526855, 2.4592208862304688, 1.888708472251892]